In [1]:
from datasets import load_dataset
from qdrant_client import QdrantClient, models
from langchain_qdrant import FastEmbedSparse, QdrantVectorStore, RetrievalMode
from qdrant_client.http.models import Distance, SparseVectorParams, VectorParams
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import re
from uuid import uuid4
import json

import math

2025-04-16 13:03:09.470424: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744808589.481717   54505 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744808589.485161   54505 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-16 13:03:09.498020: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
tqa_dataset = load_dataset("mandarjoshi/trivia_qa" , "rc")

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/24 [00:00<?, ?it/s]

In [3]:
tqa_dataset_train = tqa_dataset["train"]

In [4]:
collection_name = "wikipedia-dot"
client = QdrantClient(url="http://192.168.2.3:6333" , timeout=999)

In [5]:
model = SentenceTransformer('nthakur/contriever-base-msmarco')

In [6]:
from itertools import islice

def batched(iterable, n):
    iterator = iter(iterable)
    while batch := list(islice(iterator, n)):
        yield batch
batch_size = 64

In [7]:
import difflib

def get_overlap(s1, s2):
    s = difflib.SequenceMatcher(None, s1, s2)
    pos_a, pos_b, size = s.find_longest_match(0, len(s1), 0, len(s2)) 
    return s1[pos_a:pos_a+size]

def is_gold_compression(text, supporting_map, threshold=0.5):
    text = text.lower().strip()
    for supporting_text in supporting_map:
        supporting_text = supporting_text.lower().strip()
        if not supporting_text or not text:
            continue
        overlap = get_overlap(supporting_text, text)
        overlap_len = len(overlap)
        if (
            (overlap_len / len(supporting_text) > threshold)
            or
            (overlap_len / len(text) > threshold)
        ):
            return True
    return False

In [8]:

# total_batches = math.ceil(len(tqa_dataset_train) / batch_size)

# document_tqa_train = []

# for batch in tqdm(batched(tqa_dataset_train, batch_size), total=total_batches, desc="dataset batches"):
#     questions = [data["question"] for data in batch]
#     answers = [data["answer"] for data in batch]

#     # Lowercased answer sets
#     answers_grouped = [
#         set(s.lower() for s in ans["aliases"] + ans["normalized_aliases"] + [ans["normalized_trainue"]])
#         for ans in answers
#     ]
#     for group in answers_grouped:
#         group_copy = {s for s in group if len(s.strip()) >= 2}
#         group.clear()
#         group.update(group_copy)


#     # Encode questions
#     queries_encode = model.encode(questions)

#     # Prepare batch search
#     search_queries = [
#         models.QueryRequest(query=query, with_payload=True, limit=100)
#         for query in queries_encode
#     ]

#     # Run batch query
#     batch_point = client.query_batch_points(collection_name=collection_name, requests=search_queries)

#     # Process results
#     for i, query in enumerate(batch_point):
#         ctxs = []
#         for point in query.points:
#             doc_id = point.payload.get("docid")
#             title = point.payload.get("title")
#             text = point.payload.get("text")
#             score = point.score

#             # Match answer aliases in text
#             matches = {
#                 (match.start(), len(match.group()))
#                 for alias in answers_grouped[i]
#                 for match in re.finditer(re.escape(alias), text.lower())
#             }

#             has_answer = len(matches) > 0

#             ctxs.append({
#                 "id": doc_id,
#                 "title": title,
#                 "text": text,
#                 "score": score,
#                 "has_answer": has_answer,
#                 "answer_occurrences": [
#                     {"start": start, "length": length} for (start, length) in matches
#                 ]
#             })

#         document_tqa_train.append({
#             "question": questions[i],
#             "answers": list(answers_grouped[i]),
#             "ctxs": ctxs
#         })

# with open("dataset/document_tqa_train.json", "w", encoding="utf-8") as f:
#     json.dump(document_tqa_train, f, ensure_ascii=False, indent=2)

In [9]:
two_wiki_qa = load_dataset("kamelliao/2wikimultihopqa")
two_wiki_qa_dataset_train = two_wiki_qa["train"]
two_wiki_qa_dataset_train = two_wiki_qa_dataset_train.filter(
    lambda example: all(title in example['context']['title'] for title in set(example['supporting_facts']['title']))
)
total_batches = math.ceil(len(two_wiki_qa_dataset_train) / batch_size)

document_two_wiki_qa_train = []

for batch in tqdm(batched(two_wiki_qa_dataset_train, batch_size), total=total_batches, desc="dataset batches"):
    questions = [data["question"] for data in batch]
    answers = [data["answer"] for data in batch]
    
    # Lowercased answer sets
    answers_grouped = [
        set(ans.lower())
        for ans in answers
    ]

    for group in answers_grouped:
        group_copy = {s for s in group if len(s.strip()) >= 2}
        group.clear()
        group.update(group_copy)

    # Encode questions
    queries_encode = model.encode(questions)

    # Prepare batch search
    search_queries = [
        models.QueryRequest(query=query, with_payload=True, limit=100)
        for query in queries_encode
    ]

    # Run batch query
    batch_point = client.query_batch_points(collection_name=collection_name, requests=search_queries)

    # Process results
    for i, query in enumerate(batch_point):
        support_titles = batch[i]["supporting_facts"]["title"]
        support_sent_ids = batch[i]["supporting_facts"]["sent_id"]
        context_titles = batch[i]["context"]["title"]
        context_sentences = batch[i]["context"]["sentences"]
        
        # Build supporting sentence map per title
        supporting_map = []
        for title, sent_id in zip(support_titles, support_sent_ids):
            try:
                title_idx = context_titles.index(title)
                sentence = context_sentences[title_idx][sent_id]
                supporting_map.append(sentence.strip())
            except (ValueError, IndexError):
                continue
        ctxs = []
        for point in query.points:
            doc_id = point.payload.get("docid")
            title = point.payload.get("title")
            text = point.payload.get("text")
            score = point.score

            # Match answer aliases in text
            matches = {
                (match.start(), len(match.group()))
                for alias in answers_grouped[i]
                for match in re.finditer(re.escape(alias), text.lower())
            }

            gold_document = is_gold_compression(text, supporting_map)

            has_answer = len(matches) > 0

            ctxs.append({
                "id": doc_id,
                "title": title,
                "text": text,
                "score": score,
                "has_answer": has_answer,
                "answer_occurrences": [
                    {"start": start, "length": length} for (start, length) in matches
                ],
                "gold_document" : gold_document
            })

        document_two_wiki_qa_train.append({
            "question": questions[i],
            "answers": list(answers_grouped[i]),
            "ctxs": ctxs
        })

with open("dataset/document_two_wiki_qa_train.json", "w", encoding="utf-8") as f:
    json.dump(document_two_wiki_qa_train, f, ensure_ascii=False, indent=2)


ataset batches: 100%|████████████████████████████████████████████████████████████| 2617/2617 [1:21:03<00:00,  1.86s/it]